In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.tree import RandomForest, RandomForestModel

from pyspark.mllib.util import MLUtils

%pylab inline

import numpy as np
from time import time

import sys,os,pickle
sys.path.insert(0, os.path.abspath('../src'))

from BoostStumps import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pyspark import SparkContext

#sc=SparkContext()

from numpy.random import rand
p=0.9
data=[]
for i in range(10):
    for j in range(10):
        if np.abs(i-4)<3 and np.abs(j-6)<3:
            y=2*(rand()<p)-1
        else:
            y=2*(rand()>p)-1
        print "%1.0f "%((1+y)/2),
        data.append(LabeledPoint(y,[i,j]))
    print

0  0  0  0  0  0  0  0  0  1 
0  0  0  0  0  0  0  0  0  0 
0  0  0  0  1  1  1  1  1  0 
0  0  0  0  1  1  1  1  1  0 
0  0  0  0  1  1  1  0  1  0 
0  0  0  0  1  1  1  1  1  0 
0  0  0  0  0  1  1  1  1  0 
1  0  0  0  0  0  0  1  0  0 
0  0  0  0  0  0  0  0  0  0 
0  0  0  0  0  0  0  0  0  0 


In [3]:
dataRDD=sc.parallelize(data,numSlices=2)
dataRDD.getNumPartitions()

dataRDD = sc.textFile("../data/covtype.data").map(lambda line: [float(x.strip()) for x in line.split(',')]).map(lambda line: LabeledPoint(float(line[-1]==1),line[:-1])).cache()
#dataRDD = dataRDD.sample(False,0.1, seed=255).cache()
dataRDD.first()

LabeledPoint(0.0, [2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [4]:
sc

In [5]:
booster=Booster(sc,dataRDD)

number of features= 54 number of partitions= 54
Sizes: Data1=581012, trainingData=406904, testData=174108
number of elements in GR= 54
number of elements in GTR= 54
number of partitions in PS= 54


In [6]:
print booster.report_times()

 20.93 : Split into train and test
  0.00 : glom
  0.00 : add partition index
 12.10 : Prepare_partition_data_structure
  0.62 : Compute Split points
  1.75 : Add_weak_learner_matrix
  0.00 : Finished Initialization


In [7]:
Scores=[]
for i in range(10):
    booster.boosting_iteration()
    Scores.append(booster.compute_scores())

In [8]:
print booster.report_times()

 20.93 : Split into train and test
  0.00 : glom
  0.00 : add partition index
 12.10 : Prepare_partition_data_structure
  0.62 : Compute Split points
  1.75 : Add_weak_learner_matrix
  0.00 : Finished Initialization
 19.84 : Start main loop 0
  0.70 : found best splitter 0
  1.97 : Updated Weights 0
  0.00 : Start main loop 1
  0.73 : found best splitter 1
  1.83 : Updated Weights 1
  0.00 : Start main loop 2
  0.78 : found best splitter 2
  1.83 : Updated Weights 2
  0.00 : Start main loop 3
  0.80 : found best splitter 3
  1.89 : Updated Weights 3
  0.00 : Start main loop 4
  0.63 : found best splitter 4
  1.95 : Updated Weights 4
  0.00 : Start main loop 5
  0.65 : found best splitter 5
  1.85 : Updated Weights 5
  0.00 : Start main loop 6
  0.68 : found best splitter 6
  1.89 : Updated Weights 6
  0.00 : Start main loop 7
  0.62 : found best splitter 7
  1.86 : Updated Weights 7
  0.00 : Start main loop 8
  0.89 : found best splitter 8
  1.96 : Updated Weights 8
  0.00 : Start main

In [9]:
booster.proposals

[{'best feature': 22,
  'details': [{'Correlation': 0.36799999999999999,
    'Feature_index': 0,
    'SS': array([-0.3656    , -0.3512    , -0.31786667, -0.28      , -0.2304    ,
           -0.15546667, -0.0296    ,  0.1088    ,  0.24906667,  0.368     ]),
    'Threshold': 1.7976931348623157e+308,
    'Threshold_index': 9},
   {'Correlation': 0.34531706022817721,
    'Feature_index': 1,
    'SS': array([-0.27739453, -0.20549217, -0.1250995 , -0.05531971,  0.00570443,
            0.05929955,  0.13014062,  0.20761475,  0.27553728,  0.34531706]),
    'Threshold': 1.7976931348623157e+308,
    'Threshold_index': 9},
   {'Correlation': 0.36408478869484068,
    'Feature_index': 2,
    'SS': array([-0.28676177, -0.2227703 , -0.1259832 , -0.07718971,  0.01479803,
            0.09212105,  0.16117851,  0.23370217,  0.31182509,  0.36408479]),
    'Threshold': 1.7976931348623157e+308,
    'Threshold_index': 9},
   {'Correlation': 0.34055010893246185,
    'Feature_index': 3,
    'SS': array([-0.3026